In [1]:
from utils.list_files import list_files
from utils.progress import progress_parallel

import ujson as json

from itertools import islice
from collections import Counter
from datetime import datetime
import os
import csv

In [2]:
os.listdir("metadata")

['parler-videos-geocoded.json']

In [15]:
def load(fn):
    with open(fn) as f:
        data = json.load(f)
    video_id = os.path.basename(fn).split('.')[0].split('-')[1]
    for i in range(len(data)):
        data[i]['tag'] = video_id
    return data

metadata = progress_parallel(load, list_files('metadata'))[0]

1 0:00:00 155344.59/s


In [16]:
import pickle
with open('metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)
    
# with open('metadata.pkl', 'rb') as f:
#     metadata = pickle.load(f)

len(metadata)

68284

In [17]:
def geocoded():
    for e in metadata:
        try:
            lon = float(e['Longitude'])
            lat = float(e['Latitude'])
            created = datetime.strptime(e['Timestamp'], '%Y-%m-%d %H:%M:%S')
            video_id = e['ID']
            
            yield(lon, lat, created, video_id)
    #   
        except KeyError:
            continue
        except ValueError:
            continue

# collect all geocoded data
collected = list(geocoded())

In [18]:
# top,left = [38.892110, -77.015188]
# bottom,right = [38.887619, -77.005959]

# dc area
top,left = [38.978921, -77.140698]
bottom,right = [38.808277, -76.914339]

start = datetime(2021, 1, 5)

In [19]:
to_download = []
for lon,lat,created,video_id in collected:
    if lon < left or lon > right or lat > top or lat < bottom:
        continue
    if created < start:
        continue
    to_download.append(video_id)
    
len(to_download)

1499

In [20]:
with open('dc-area.txt', 'w') as f:
    for e in to_download:
        f.write(f'https://video.parler.com/{e[:2]}/{e[2:4]}/{e}.mp4\n')